In [ ]:
#%pip install scikit-learn --upgrade --user

In [ ]:

from builtins import print
import numpy as np
import pandas as pd
import matplotlib
import random

matplotlib.use('agg')
import matplotlib.pyplot as plt

matplotlib.rcParams['font.family'] = 'sans-serif'
matplotlib.rcParams['font.sans-serif'] = 'Arial'

import os
import operator
#import utils

#from utils.constants import UNIVARIATE_DATASET_NAMES as DATASET_NAMES
#from utils.constants import UNIVARIATE_ARCHIVE_NAMES as ARCHIVE_NAMES

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder

import keras
import tensorflow as tf
#from utils.utils import calculate_metrics
#from utils.utils import create_directory
#from utils.utils import check_if_file_exits
import gc
import time

#from utils.utils import save_logs
#from utils.utils import save_test_duration

#from utils.utils import read_all_datasets
#from utils.utils import transform_labels
#from utils.utils import create_directory
#from utils.utils import run_length_xps
#from utils.utils import generate_results_csv

import sys
import sklearn

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir("/content/drive/MyDrive/Colab Notebooks/Candidas/archives/TSC/AllCandidasBload/")
!ls

AllCandidasBload_TEST.csv  AllCandidasBload_TRAIN.csv  AllCandidasBload_VAL.csv


In [ ]:
open("/content/drive/MyDrive/Colab Notebooks/Candidas/archives/TSC/AllCandidasBload/AllCandidasBload_TRAIN.csv")

<_io.TextIOWrapper name='/content/drive/MyDrive/Colab Notebooks/Candidas/archives/TSC/AllCandidasBload/AllCandidasBload_TRAIN.csv' mode='r' encoding='UTF-8'>

In [ ]:
# SIZE_FILE = 652
SIZE_FILE = 512

In [ ]:
def check_if_file_exits(file_name):
    return os.path.exists(file_name)


def readucr(filename, delimiter=','):
    print(filename)
    data = pd.read_csv(filename, delimiter=delimiter,header=None)
    Y = data.iloc[:, SIZE_FILE].values
    X = data.iloc[:,0: SIZE_FILE].copy().values
    return X, Y


def readsits(filename, delimiter=','):
    data = np.genfromtxt(filename, delimiter=delimiter)
    Y = data[:, -1]
    X = data[:, :-1]
    return X, Y

In [ ]:
def create_directory(directory_path):
    if os.path.exists(directory_path):
        return None
    else:
        try:
            os.makedirs(directory_path)
        except:
            # in case another machine created the path meanwhile !:(
            return None
        return directory_path

In [ ]:
def read_dataset(root_dir, archive_name, dataset_name):
    datasets_dict = {}

    file_name = root_dir + '/archives/' + archive_name + '/' + dataset_name + '/' + dataset_name
    x_train, y_train = readucr(file_name + '_TRAIN.csv')
    #Trocar aqui para teste depois
    x_test, y_test = readucr(file_name + '_TEST.csv')
    x_val, y_val = readucr(file_name + '_VAL.csv')
    print(f"x_train value:{x_train}")
    print(f"y_train value:{y_train}")
    print(f"x_test value:{x_test}")
    print(f"y_test value:{y_test}")
    print(f"x_val value:{x_val}")
    print(f"y_val value:{y_val}")

    datasets_dict[dataset_name] = (x_train.copy(), y_train.copy(), x_test.copy(),
                                   y_test.copy(), x_val.copy(),y_val.copy())

    return datasets_dict

In [ ]:
def read_all_datasets(root_dir, archive_name):
    datasets_dict = {}

    dataset_names_to_sort = []

    if archive_name == 'TSC':
        for dataset_name in UNIVARIATE_DATASET_NAMES:
            root_dir_dataset = root_dir + '/archives/' + archive_name + '/' + dataset_name + '/'
            file_name = root_dir_dataset + dataset_name
            print(file_name)
            x_train, y_train = readucr(file_name + '_TRAIN.csv')
            #Trocar aqui para teste depois
            x_test, y_test = readucr(file_name + '_TEST.csv')
            x_val, y_val = readucr(file_name + '_VAL.csv')
            print(f"Valor do Xtrain no read_all_datasets {x_train}")
            print(f"Valor do y_train no read_all_datasets {y_train}")
            datasets_dict[dataset_name] = (x_train.copy(), y_train.copy(), x_test.copy(),
                                           y_test.copy(),x_val.copy(),y_val.copy())

            dataset_names_to_sort.append((dataset_name, len(x_train)))

        dataset_names_to_sort.sort(key=operator.itemgetter(1))

        for i in range(len(UNIVARIATE_DATASET_NAMES)):
            UNIVARIATE_DATASET_NAMES[i] = dataset_names_to_sort[i][0]
    return datasets_dict

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix
def specificity_score(y_true, y_pred):
    cm  = multilabel_confusion_matrix(y_true, y_pred)
    specificity = []
    for i in range(len(cm)):
        tn = cm[i][0][0]
        fp = cm[i][0][1]
        spec = tn / (tn + fp)
        specificity.append(spec)
    avg_specificity = sum(specificity) / len(specificity)
    return avg_specificity

In [ ]:
precision = []
accuracy = []
recall = []
f1 = []
specificity = []
std = 0

def calculate_metrics(y_test, y_pred, duration):
    res = pd.DataFrame(data=np.zeros((1, 7), dtype=np.double), index=[0],
                       columns=['precision', 'accuracy', 'recall','f1_score','duration','std','specificity'])
    temp = precision_score(y_test, y_pred, average='macro')
    print(f"Temp: {temp}")
    res['precision'] = temp
    res['accuracy'] = accuracy_score(y_test, y_pred)
    res['recall'] = recall_score(y_test, y_pred, average='macro')
    res['f1_score'] = f1_score(y_test, y_pred, average='macro')
    res['specificity'] = specificity_score(y_test, y_pred)
    res['duration'] = duration
    print(res)
    return res

def calculate_metrics_per_class(y_val, y_pred, classes):
    from sklearn.metrics import confusion_matrix
    print(f"Classes before: {classes}")
    classes = np.unique(classes)  # ou np.arange(n_classes) se souber o número de classes
    print(f"Classes after: {classes}")
    y_pred = np.argmax(y_pred, axis=1)
    y_val = np.argmax(y_val, axis=1)
    print(f"Y_pred: {y_pred}")
    print(f"y_val {y_val}")
    cm = confusion_matrix(y_val, y_pred, labels=classes)
    metrics_per_class = []

    for i, class_label in enumerate(classes):
        TP = cm[i, i]
        FN = cm[i, :].sum() - TP
        FP = cm[:, i].sum() - TP
        TN = cm.sum() - (TP + FP + FN)

        accuracy = (TP + TN) / cm.sum() if cm.sum() > 0 else 0
        precision = TP / (TP + FP) if (TP + FP) > 0 else 0
        recall = TP / (TP + FN) if (TP + FN) > 0 else 0
        specificity = TN / (TN + FP) if (TN + FP) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        metrics_per_class.append({
            'class': class_label,
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1_score': f1,
            'specificity': specificity
        })

    return metrics_per_class

In [ ]:
def save_test_duration(file_name, test_duration):
    res = pd.DataFrame(data=np.zeros((1, 1), dtype=np.double), index=[0],
                       columns=['test_duration'])
    res['test_duration'] = test_duration
    res.to_csv(file_name, index=False)

In [ ]:
def transform_labels(y_train, y_test, y_val):
    """
    Transform label to min equal zero and continuous
    For example if we have [1,3,4] --->  [0,1,2]
    """
    # no validation split
    # init the encoder
    encoder = LabelEncoder()
    # concat train and test to fit
    y_train_test_val = np.concatenate((y_train, y_test, y_val), axis=0)
    # fit the encoder
    encoder.fit(y_train_test_val)
    # transform to min zero and continuous labels
    new_y_train_test = encoder.transform(y_train_test_val)
    # resplit the train and test
    new_y_train = new_y_train_test[0:len(y_train)]
    new_y_test = new_y_train_test[len(y_train):len(y_train)+len(y_test)]
    new_y_val = new_y_train_test[len(y_train)+len(y_test):]
    return new_y_train, new_y_test, new_y_val

In [ ]:
def generate_results_csv(output_file_name, root_dir, clfs):
    res = pd.DataFrame(data=np.zeros((0, 10), dtype=np.float64), index=[],
                       columns=['classifier_name', 'archive_name', 'dataset_name', 'iteration',
                                'precision', 'accuracy', 'recall','f1_score' ,'duration','std'])
    for archive_name in UNIVARIATE_ARCHIVE_NAMES:
        datasets_dict = read_all_datasets(root_dir, archive_name)
        for classifier_name in clfs:
            durr = 0.0

            curr_archive_name = archive_name
            for dataset_name in datasets_dict.keys():
                output_dir = root_dir + '/results/' + classifier_name + '/' \
                             + curr_archive_name + '/' + dataset_name + '/' + 'df_metrics.csv'
                #print(output_dir)
                if not os.path.exists(output_dir):
                    continue
                df_metrics = pd.read_csv(output_dir)
                df_metrics['classifier_name'] = classifier_name
                df_metrics['archive_name'] = archive_name
                df_metrics['dataset_name'] = dataset_name
                df_metrics['iteration'] = 0
                res = pd.concat((res, df_metrics), axis=0, sort=False)
                durr += df_metrics['duration'][0]

    res.to_csv(root_dir + output_file_name, index=False)

    res = res.loc[res['classifier_name'].isin(clfs)]

    return res

In [ ]:
def plot_epochs_metric(hist, file_name, metric='loss'):
    plt.figure()
    plt.plot(hist.history[metric])
    plt.plot(hist.history['val_' + metric])
    plt.title('model ' + metric)
    plt.ylabel(metric, fontsize='large')
    plt.xlabel('epoch', fontsize='large')
    plt.legend(['train', 'val'], loc='upper left')
    plt.savefig(file_name, bbox_inches='tight')
    plt.close()

In [ ]:
def save_logs(output_directory, hist, y_pred, y_test, duration,
              lr=True, plot_test_acc=True):
    hist_df = pd.DataFrame(hist.history)
    #2 * (Precision * Recall) / (Precision + Recall)
    print("Histórico")
    print(hist_df)
    hist_df['f1'] = 2 * ((hist_df.iloc[:, 3] * hist_df.iloc[:, 2])/(hist_df.iloc[:, 3] + hist_df.iloc[:, 2]))
    hist_df.to_csv(output_directory + 'history.csv', index=False)

    df_metrics = calculate_metrics(y_test, y_pred, duration)
    precision.append(df_metrics['precision'])
    specificity.append(df_metrics['specificity'])
    accuracy.append(df_metrics['accuracy'])
    recall.append(df_metrics['recall'])
    f1.append(df_metrics['f1_score'])

    df_metrics.to_csv(output_directory + 'df_metrics_val.csv', index=False)

    index_best_model = hist_df['loss'].idxmin()
    row_best_model = hist_df.loc[index_best_model]

    df_best_model = pd.DataFrame(data=np.zeros((1, 6), dtype=np.float64), index=[0],
                                 columns=['best_model_train_loss', 'best_model_val_loss', 'best_model_train_acc',
                                          'best_model_val_acc', 'best_model_learning_rate', 'best_model_nb_epoch'])

    df_best_model['best_model_train_loss'] = row_best_model['loss']
    #if plot_test_acc:
    #    df_best_model['best_model_val_loss'] = row_best_model['val_loss']
    #df_best_model['best_model_train_acc'] = row_best_model['acc']
    if plot_test_acc:
        df_best_model['best_model_val_acc'] = row_best_model['val_acc']
    if lr == True:
        df_best_model['best_model_learning_rate'] = row_best_model['lr']
    df_best_model['best_model_nb_epoch'] = index_best_model

    df_best_model.to_csv(output_directory + 'df_best_model.csv', index=False)

    if plot_test_acc:
        # plot losses
        plot_epochs_metric(hist, output_directory + 'epochs_loss.png')

    return df_metrics

In [ ]:
def create_synthetic_dataset(pattern_len=[0.25], pattern_pos=[0.1, 0.65], ts_len=128, ts_n=128):
    random.seed(1234)
    np.random.seed(1234)

    nb_classes = len(pattern_pos) * len(pattern_len)

    out_dir = '/b/home/uha/hfawaz-datas/dl-tsc/archives/UCRArchive_2018/BinaryData/'

    create_directory(out_dir)

    x_train = np.random.normal(0.0, 0.1, size=(ts_n, ts_len))
    x_test = np.random.normal(0.0, 0.1, size=(ts_n, ts_len))

    y_train = np.random.randint(low=0, high=nb_classes, size=(ts_n,))
    y_test = np.random.randint(low=0, high=nb_classes, size=(ts_n,))

    # make sure at least each class has one example
    y_train[:nb_classes] = np.arange(start=0, stop=nb_classes, dtype=np.int32)
    y_test[:nb_classes] = np.arange(start=0, stop=nb_classes, dtype=np.int32)

    # each class is defined with a certain combination of pattern_pos and pattern_len
    # with one pattern_len and two pattern_pos we can create only two classes
    # example:  class 0 _____-_  & class 1 _-_____

    # create the class definitions
    class_def = [None for i in range(nb_classes)]

    idx_class = 0
    for pl in pattern_len:
        for pp in pattern_pos:
            class_def[idx_class] = {'pattern_len': int(pl * ts_len),
                                    'pattern_pos': int(pp * ts_len)}
            idx_class += 1

    # create the dataset
    for i in range(ts_n):
        # for the train
        c = y_train[i]
        curr_pattern_pos = class_def[c]['pattern_pos']
        curr_pattern_len = class_def[c]['pattern_len']
        x_train[i][curr_pattern_pos:curr_pattern_pos + curr_pattern_len] = \
            x_train[i][curr_pattern_pos:curr_pattern_pos + curr_pattern_len] + 1.0

        # for the test
        c = y_test[i]
        curr_pattern_pos = class_def[c]['pattern_pos']
        curr_pattern_len = class_def[c]['pattern_len']
        x_test[i][curr_pattern_pos:curr_pattern_pos + curr_pattern_len] = \
            x_test[i][curr_pattern_pos:curr_pattern_pos + curr_pattern_len] + 1.0

    # znorm
    x_train = (x_train - x_train.mean(axis=1, keepdims=True)) \
              / x_train.std(axis=1, keepdims=True)

    x_test = (x_test - x_test.mean(axis=1, keepdims=True)) \
             / x_test.std(axis=1, keepdims=True)

    # visualize example
    # plt.figure()
    # colors = generate_array_of_colors(nb_classes)
    # for c in range(nb_classes):
    #     plt.plot(x_train[y_train == c][0], color=colors[c], label='class-' + str(c))
    # plt.legend(loc='best')
    # plt.savefig('out.pdf')
    # exit()

    # np.save(out_dir+'x_train.npy',x_train)
    # np.save(out_dir+'y_train.npy',y_train)
    # np.save(out_dir+'x_test.npy',x_test)
    # np.save(out_dir+'y_test.npy',y_test)

    # print('Done creating dataset!')

    return x_train, y_train, x_test, y_test

In [ ]:
def generate_array_of_colors(n):
    # https://www.quora.com/How-do-I-generate-n-visually-distinct-RGB-colours-in-Python
    ret = []
    r = int(random.random() * 256)
    g = int(random.random() * 256)
    b = int(random.random() * 256)
    alpha = 1.0
    step = 256 / n
    for i in range(n):
        r += step
        g += step
        b += step
        r = int(r) % 256
        g = int(g) % 256
        b = int(b) % 256
        ret.append((r / 255, g / 255, b / 255, alpha))
    return ret

In [ ]:
# def read_sits_xps(root_dir):
#     datasets_dict = {}
#     path_to_data = root_dir + 'archives/SITS/resampled-SITS/'
#     path_to_test = root_dir + 'archives/SITS/' + 'SatelliteFull_TEST_1000.csv'

#     x_test, y_test = readsits(path_to_test)

#     for subdir, dirs, files in os.walk(path_to_data):
#         for file_name in files:
#             arr = file_name.split('.')
#             dataset_name = arr[0]
#             file_type = arr[1]
#             if file_type == 'csv':
#                 x_train, y_train = readsits(subdir + '/' + file_name)

#                 datasets_dict[dataset_name] = (x_train.copy(), y_train.copy(), x_test.copy(),
#                                                y_test.copy())

#     return datasets_dict

In [ ]:
def resample_dataset(x, rate):
    new_x = np.zeros(shape=(x.shape[0], rate))
    from scipy import signal
    for i in range(x.shape[0]):
        f = signal.resample(x[0], rate)
        new_x[i] = f
    return new_x

In [ ]:
def run_length_xps(root_dir):
    #archive_name = ARCHIVE_NAMES[0]
    archive_name = UNIVARIATE_ARCHIVE_NAMES[0]
    dataset_name = 'InlineSkate'
    datasets_dict = read_dataset(root_dir, archive_name, dataset_name)

    lengths = [2 ** i for i in range(5, 12)]

    x_train = datasets_dict[dataset_name][0]
    y_train = datasets_dict[dataset_name][1]
    x_test = datasets_dict[dataset_name][2]
    y_test = datasets_dict[dataset_name][3]
    x_val = datasets_dict[dataset_name][4]
    y_val = datasets_dict[dataset_name][5]

    new_archive_name = 'TSC'
#    new_archive_name = 'InlineSkateXPs'

    for l in lengths:
        new_x_train = resample_dataset(x_train, l)
        new_x_test = resample_dataset(x_test, l)
        new_x_val = resample_dataset(x_val, l)
        new_dataset_name = dataset_name + '-' + str(l)
        new_dataset_dir = root_dir + 'archives/' + new_archive_name + '/' + new_dataset_name + '/'
        create_directory(new_dataset_dir)

        np.save(new_dataset_dir + 'x_train.npy', new_x_train)
        np.save(new_dataset_dir + 'y_train.npy', y_train)
        np.save(new_dataset_dir + 'x_test.npy', new_x_test)
        np.save(new_dataset_dir + 'y_test.npy', y_test)
        np.save(new_dataset_dir + 'x_val.npy', new_x_val)
        np.save(new_dataset_dir + 'y_val.npy', y_val)

In [ ]:
UNIVARIATE_DATASET_NAMES = ['AllCandidasBload']

UNIVARIATE_ARCHIVE_NAMES = ['TSC', 'InlineSkateXPs', 'SITS']

SITS_DATASETS = ['SatelliteFull_TRAIN_c301', 'SatelliteFull_TRAIN_c200', 'SatelliteFull_TRAIN_c451',
                 'SatelliteFull_TRAIN_c89', 'SatelliteFull_TRAIN_c677', 'SatelliteFull_TRAIN_c59',
                 'SatelliteFull_TRAIN_c133']

InlineSkateXPs_DATASETS = ['InlineSkate-32', 'InlineSkate-64', 'InlineSkate-128',
                           'InlineSkate-256', 'InlineSkate-512', 'InlineSkate-1024',
                           'InlineSkate-2048']

dataset_names_for_archive = {'TSC': UNIVARIATE_DATASET_NAMES,
                             'SITS': SITS_DATASETS,
                             'InlineSkateXPs': InlineSkateXPs_DATASETS}

In [ ]:
class Classifier_NNE:

    def create_classifier(self, model_name, input_shape, nb_classes, output_directory, verbose=False,
                          build=True):
        if self.check_if_match('inception*', model_name):
            #from classifiers import inception
            return Classifier_INCEPTION(output_directory, input_shape, nb_classes, verbose, build=build)

    def check_if_match(self, rex, name2):
        import re
        pattern = re.compile(rex)
        return pattern.match(name2)

    def __init__(self, output_directory, input_shape, nb_classes, verbose=False, nb_iterations=5,
                 clf_name='inception'):
        self.classifiers = [clf_name]
        out_add = ''
        for cc in self.classifiers:
            out_add = out_add + cc + '-'
        self.archive_name = UNIVARIATE_ARCHIVE_NAMES[0]
        self.iterations_to_take = [i for i in range(nb_iterations)]
        for cc in self.iterations_to_take:
            out_add = out_add + str(cc) + '-'
        self.output_directory = output_directory.replace('nne',
                                                         'nne' + '/' + out_add)
        create_directory(self.output_directory)
        self.dataset_name = output_directory.split('/')[-2]
        self.verbose = verbose
        self.models_dir = output_directory.replace('nne', 'classifier')

    def fit(self, x_train, y_train, x_test, y_test, y_true):
        # no training since models are pre-trained
        start_time = time.time()

        y_pred = np.zeros(shape=y_test.shape)

        ll = 0

        # loop through all classifiers
        for model_name in self.classifiers:
            # loop through different initialization of classifiers
            for itr in self.iterations_to_take:
                if itr == 0:
                    itr_str = ''
                else:
                    itr_str = '_itr_' + str(itr)

                curr_archive_name = self.archive_name + itr_str

                curr_dir = self.models_dir.replace('classifier', model_name).replace(
                    self.archive_name, curr_archive_name)

                model = self.create_classifier(model_name, None, None,
                                               curr_dir, build=False)

                predictions_file_name = curr_dir + 'y_pred.npy'
                # check if predictions already made
                if check_if_file_exits(predictions_file_name):
                    # then load only the predictions from the file
                    curr_y_pred = np.load(predictions_file_name)
                else:
                    # then compute the predictions
                    curr_y_pred = model.predict(x_test, y_true, x_train, y_train, y_test,
                                                return_df_metrics=False)
                    keras.backend.clear_session()

                    np.save(predictions_file_name, curr_y_pred)

                y_pred = y_pred + curr_y_pred

                ll += 1

        # average predictions
        y_pred = y_pred / ll

        # save predictions
        np.save(self.output_directory + 'y_pred.npy', y_pred)

        # convert the predicted from binary to integer
        y_pred = np.argmax(y_pred, axis=1)

        duration = time.time() - start_time

        df_metrics = calculate_metrics(y_test, y_pred, duration)

        df_metrics.to_csv(self.output_directory + 'df_metrics_val.csv', index=False)

        gc.collect()

In [ ]:
class Classifier_INCEPTION:

    def __init__(self, output_directory, input_shape, nb_classes, iter, verbose=False, inverbose=False, build=True, batch_size=64,
                 nb_filters=32, use_residual=True, use_bottleneck=True, depth=6, kernel_size=41, nb_epochs=1500):

        self.output_directory = output_directory

        self.nb_filters = nb_filters
        self.use_residual = use_residual
        self.use_bottleneck = use_bottleneck
        self.depth = depth
        self.kernel_size = kernel_size - 1
        self.callbacks = None
        self.batch_size = batch_size
        self.bottleneck_size = 32
        self.nb_epochs = nb_epochs
        self.all_class_metrics = []
        self.iter = iter


        if build == True:
            self.model = self.build_model(input_shape, nb_classes)
            if (verbose == True):
                self.model.summary()
            self.verbose = verbose
            self.model.save_weights(self.output_directory + 'model_init.weights.h5')

    def _inception_module(self, input_tensor, stride=1, activation='linear'):

        if self.use_bottleneck and int(input_tensor.shape[-1]) > 1:
            input_inception = keras.layers.Conv1D(filters=self.bottleneck_size, kernel_size=1,
                                                  padding='same', activation=activation, use_bias=False)(input_tensor)
        else:
            input_inception = input_tensor

        # kernel_size_s = [3, 5, 8, 11, 17]
        kernel_size_s = [self.kernel_size // (2 ** i) for i in range(3)]

        conv_list = []

        for i in range(len(kernel_size_s)):
            conv_list.append(keras.layers.Conv1D(filters=self.nb_filters, kernel_size=kernel_size_s[i],
                                                 strides=stride, padding='same', activation=activation, use_bias=False)(
                input_inception))

        max_pool_1 = keras.layers.MaxPool1D(pool_size=3, strides=stride, padding='same')(input_tensor)

        conv_6 = keras.layers.Conv1D(filters=self.nb_filters, kernel_size=1,
                                     padding='same', activation=activation, use_bias=False)(max_pool_1)

        conv_list.append(conv_6)

        x = keras.layers.Concatenate(axis=2)(conv_list)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Activation(activation='relu')(x)
        return x

    def _shortcut_layer(self, input_tensor, out_tensor):
        shortcut_y = keras.layers.Conv1D(filters=int(out_tensor.shape[-1]), kernel_size=1,
                                         padding='same', use_bias=False)(input_tensor)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

        x = keras.layers.Add()([shortcut_y, out_tensor])
        x = keras.layers.Activation('relu')(x)
        return x

    def build_model(self, input_shape, nb_classes):
        input_layer = keras.layers.Input(input_shape)

        x = input_layer
        input_res = input_layer

        for d in range(self.depth):

            x = self._inception_module(x)

            if self.use_residual and d % 3 == 2:
                x = self._shortcut_layer(input_res, x)
                input_res = x

        gap_layer = keras.layers.GlobalAveragePooling1D()(x)

        output_layer = keras.layers.Dense(nb_classes, activation='softmax')(gap_layer)

        model = keras.models.Model(inputs=input_layer, outputs=output_layer)

        model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(),
                      metrics= [
                                tf.keras.metrics.CategoricalAccuracy(name='accuracy', dtype=None),
                                tf.keras.metrics.Recall(),
                                tf.keras.metrics.Precision(),
                                tf.keras.metrics.TruePositives(),
                                tf.keras.metrics.TrueNegatives(),
                                tf.keras.metrics.FalsePositives(),
                                tf.keras.metrics.FalseNegatives()
                                ])

        reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.5, patience=50,
                                                      min_lr=0.0001)

        file_path = self.output_directory + 'best_model.keras'

        model_checkpoint = keras.callbacks.ModelCheckpoint(filepath=file_path, monitor='loss',
                                                           save_best_only=True)

        self.callbacks = [reduce_lr, model_checkpoint]

        return model

    def fit(self, x_train, y_train, x_val, y_val, y_true, plot_test_acc=False):
        #if len(keras.backend.tensorflow_backend._get_available_gpus()) == 0:
        #    print('error no gpu')
        #    exit()
        # x_val and y_val are only used to monitor the test loss and NOT for training
        print("Dentro do FitModel")
        if self.batch_size is None:
            mini_batch_size = int(min(x_train.shape[0] / 10, 16))
        else:
            mini_batch_size = self.batch_size

        start_time = time.time()

        if plot_test_acc:

            hist = self.model.fit(x_train, y_train, batch_size=mini_batch_size, epochs=self.nb_epochs,
                                  verbose=self.verbose, validation_data=(x_val, y_val), callbacks=self.callbacks)
        else:

            hist = self.model.fit(x_train, y_train, batch_size=mini_batch_size, epochs=self.nb_epochs,
                                  verbose=self.verbose, callbacks=self.callbacks)

        duration = time.time() - start_time

        self.model.save(self.output_directory + 'last_model.hdf5')

        y_pred = self.predict(x_val, y_true, x_train, y_train, y_val,
                              return_df_metrics=False)

        # save predictions
        np.save(self.output_directory + 'y_pred.npy', y_pred)
         # save predictions
        np.save(self.output_directory + 'y_true.npy', y_true)

        # convert the predicted from binary to integer
        print(f"y_pred before save =  \n {y_pred}")
        y_pred = np.argmax(y_pred, axis=1)

        df_metrics = save_logs(self.output_directory, hist, y_pred, y_true, duration,
                               plot_test_acc=plot_test_acc, lr=False)

        keras.backend.clear_session()

        return df_metrics

    def predict(self, x_test, y_true, x_train, y_train, y_test, return_df_metrics=True):
        start_time = time.time()
        model_path = self.output_directory + 'best_model.keras'
        model = keras.models.load_model(model_path)
        y_pred = model.predict(x_test, batch_size=self.batch_size)
        test_duration = time.time() - start_time
        save_test_duration(self.output_directory + 'test_duration.csv', test_duration)

        classes = np.argmax(y_train, axis=1)
        class_metrics = calculate_metrics_per_class(y_test, y_pred, classes)
        for metrics in class_metrics:
            metrics.update({'model': "IncepionTime", 'fold': self.iter})
            self.all_class_metrics.append(metrics)
        df = pd.DataFrame(self.all_class_metrics)
        df.to_csv(self.output_directory + "all_class_metrics.csv", index=False)

        if return_df_metrics:
            y_pred = np.argmax(y_pred, axis=1)
            df_metrics = calculate_metrics(y_test, y_pred, 0.0)
            return df_metrics
        else:
            return y_pred

In [ ]:
def prepare_data():
    x_train = datasets_dict[dataset_name][0]
    y_train = datasets_dict[dataset_name][1]
    x_test = datasets_dict[dataset_name][2]
    y_test = datasets_dict[dataset_name][3]
    x_val = datasets_dict[dataset_name][4]
    y_val = datasets_dict[dataset_name][5]
    print(f"valor de x_train no prepare data {x_train}")
    print(f"valor de y_train no prepare data {y_train}")
    print(f"valor de y_val no prepare data {y_val}")
    nb_classes = len(np.unique(np.concatenate((y_train, y_test,y_val ), axis=0)))

    # make the min to zero of labels
    y_train, y_test, y_val = transform_labels(y_train, y_test, y_val)

    # save orignal y because later we will use binary
    y_true = y_test.astype(np.int64)
    y_true_train = y_train.astype(np.int64)
    y_true_val = y_val.astype(np.int64)
    # transform the labels from integers to one hot vectors
    enc = sklearn.preprocessing.OneHotEncoder()
    enc.fit(np.concatenate((y_train, y_test,y_val), axis=0).reshape(-1, 1))
    y_train = enc.transform(y_train.reshape(-1, 1)).toarray()
    y_test = enc.transform(y_test.reshape(-1, 1)).toarray()
    y_val = enc.transform(y_val.reshape(-1, 1)).toarray()

    if len(x_train.shape) == 2:  # if univariate
        # add a dimension to make it multivariate with one dimension
        x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))
        x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], 1))
        x_val = x_val.reshape((x_val.shape[0], x_val.shape[1], 1))

    return x_train, y_train, x_test, y_test, x_test, x_val, y_val, y_true, nb_classes, y_true_train, enc

In [ ]:
def fit_classifier(iter):
    input_shape = x_train.shape[1:]

    classifier = create_classifier(classifier_name, input_shape, nb_classes,
                                   output_directory, iter)

    classifier.fit(x_train, y_train, x_test, y_test, y_true)

In [ ]:
def create_classifier(classifier_name, input_shape, nb_classes, output_directory, iter,
                      verbose=False, build=True):
    if classifier_name == 'nne':
        #from classifiers import nne
        return Classifier_NNE(output_directory, input_shape, nb_classes, verbose)

    if classifier_name == 'inception':
        #from classifiers import inception
        return Classifier_INCEPTION(output_directory, input_shape, nb_classes, iter,
                                    verbose, build=build)

In [ ]:
def get_xp_val(xp):
    if xp == 'batch_size':
        xp_arr = [16, 32, 128]
    elif xp == 'use_bottleneck':
        xp_arr = [False]
    elif xp == 'use_residual':
        xp_arr = [False]
    elif xp == 'nb_filters':
        xp_arr = [16, 64]
    elif xp == 'depth':
        xp_arr = [3, 9]
    elif xp == 'kernel_size':
        xp_arr = [8, 64]
    else:
        raise Exception('wrong argument')
    return xp_arr

In [ ]:
############################################### main

#root_dir = '/b/home/uha/hfawaz-datas/temp-dl-tsc/'

root_dir = '/content/drive/MyDrive/Colab Notebooks/Candidas'

xps = ['use_bottleneck', 'use_residual', 'nb_filters', 'depth',
       'kernel_size', 'batch_size']

In [ ]:
import tensorflow as tf

In [ ]:
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

In [ ]:
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   print(
#       '\n\nThis error most likely means that this notebook is not '
#       'configured to use a GPU.  Change this in Notebook Settings via the '
#       'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
#   raise SystemError('GPU device not found')

In [ ]:
# run nb_iter_ iterations of Inception on the whole TSC archive
classifier_name = 'inception'
#archive_name = ARCHIVE_NAMES[0]
archive_name = UNIVARIATE_ARCHIVE_NAMES[0]
nb_iter_ = 10

datasets_dict = read_all_datasets(root_dir, archive_name)

for iter in range(nb_iter_):
    print('\t\titer', iter)

    trr = ''
    if iter != 0:
        trr = '_itr_' + str(iter)

    tmp_output_directory = root_dir + '/results/' + classifier_name + '/' + archive_name + trr + '/'

    for dataset_name in dataset_names_for_archive[archive_name]:
        print('\t\t\tdataset_name: ', dataset_name)

        x_train, y_train, x_test, y_test, x_test, x_val, y_val, y_true, nb_classes, y_true_train, enc = prepare_data()
        # print(f"Valor de y_train: {y_train}")
        # print(f"Valor de x_train: {x_train}")


        output_directory = tmp_output_directory + dataset_name + '/'
        print(f"output_directory: {output_directory}")
        temp_output_directory = create_directory(output_directory)

        if temp_output_directory is None:
            print('Already_done', tmp_output_directory, dataset_name)
            continue

        try:
            with tf.device('/device:GPU:0'):
                fit_classifier(iter)
                print('\t\t\t\tDONE')
        except:
            fit_classifier(iter)
            print('\t\t\t\tDONE')

            # the creation of this directory means
            create_directory(output_directory + '/DONE')

std = np.std(accuracy)
root_dir = '/content/drive/MyDrive/Colab Notebooks/Candidas'

clf_name = 'inception'

path = '/'

def update_metrics(root_dir,path ,archive_name, dataset_name, std):
    output_dir = root_dir + path + archive_name + '/' \
                            + dataset_name + '/' + 'df_metrics.csv'
    #print(output_dir)
    #print('j = ',j,'i = ',i)
    if os.path.exists(output_dir):
        df_metrics = pd.read_csv(output_dir)
        df_metrics['std'] = std
        df_metrics.to_csv(output_dir + 'df_metrics_val.csv', index=False)
    return df_metrics

/content/drive/MyDrive/Colab Notebooks/Candidas/archives/TSC/AllCandidasBload/AllCandidasBload
/content/drive/MyDrive/Colab Notebooks/Candidas/archives/TSC/AllCandidasBload/AllCandidasBload_TRAIN.csv
/content/drive/MyDrive/Colab Notebooks/Candidas/archives/TSC/AllCandidasBload/AllCandidasBload_TEST.csv
/content/drive/MyDrive/Colab Notebooks/Candidas/archives/TSC/AllCandidasBload/AllCandidasBload_VAL.csv
Valor do Xtrain no read_all_datasets [[1295.   843.3 1091.  ...  846.4 1120.   497.5]
 [1486.   983.3 1471.  ...  984.7 1659.   578.5]
 [1374.   927.9  561.3 ...  915.  1371.   559.5]
 ...
 [1255.   813.1 1096.  ...  815.5 1091.   472.9]
 [1571.  1023.  1555.  ... 1027.  1561.   626.2]
 [1551.  1053.  1329.  ... 1060.  1319.   621.6]]
Valor do y_train no read_all_datasets [5. 2. 6. 1. 4. 6. 1. 6. 2. 5. 1. 6. 1. 5. 1. 1. 6. 4. 5. 6. 6. 6. 5. 4.
 2. 3. 5. 5. 1. 2. 6. 6. 2. 1. 5. 6. 4. 4. 2. 5. 5. 4. 6. 6. 4. 6. 2. 5.
 6. 2. 1. 6. 6. 5. 2. 6. 5. 1. 5. 5. 6. 4. 5. 4. 5. 1. 5. 2. 4. 2. 5. 5.

2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 2s/step
Classes before: [4 1 5 0 3 5 0 5 1 4 0 5 0 4 0 0 5 3 4 5 5 5 4 3 1 2 4 4 0 1 5 5 1 0 4 5 3
 3 1 4 4 3 5 5 3 5 1 4 5 1 0 5 5 4 1 5 4 0 4 4 5 3 4 3 4 0 4 1 3 1 4 4 1 1
 3 4 1 4 1 0 3 5 1 5 4 2 3 5 1 0 1 0 5 3 5 5 5 1 4 1 1 1 0 1 1 0 2 5 5 0 1
 4 4 5 4 1 4 4 5 5 0 3 1 1 5 0 0 5 0 0 5 1 5 5 1 3 1 5 0 0 4 4 4 5 0 4 1 0
 0 1 4 1 1 1 5 2 1 1 5 5 4 4 4 0 5 1 4 3 2 5 5 4 4 3 5 0 1 5 3 5 3 1 0 5 1
 3 5 2 1 1 4 3 0 5 5 2 0 0 1 0 1 4 1 3]
Classes after: [0 1 2 3 4 5]
Y_pred: [5 4 5 1 0 5 1 3 3 5 1 2 1 1 0 1 4 3 5 5 5 1 4 4 1 4 5 1 4 1 3 4 0 0 0 5 5
 4 1 1 2 0 1 3 4 4 1 1 3 5 3 5 5 4 1 5 0 1 1 0 5 3 0 5 1 1 1 1]
y_val [5 4 5 1 0 5 1 3 3 5 1 2 1 1 0 1 1 3 5 5 5 1 4 4 1 4 5 1 4 1 0 4 0 0 0 5 5
 4 1 1 2 0 1 3 4 4 1 1 3 5 3 5 5 4 1 5 0 1 1 0 5 3 0 5 1 1 1 1]
y_pred before save =  
 [[5.74094011e-05 6.50419815e-06 7.67592780e-07 8.28958309e-06
  2.28758654e-06 9.99924779e-01]
 [2.94819038e-05 1.48731806e-06 4.86048202e-06 5.21335518e-04
  9.99153614e-01 2.89170595e-04

2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 901ms/step
Classes before: [4 1 5 0 3 5 0 5 1 4 0 5 0 4 0 0 5 3 4 5 5 5 4 3 1 2 4 4 0 1 5 5 1 0 4 5 3
 3 1 4 4 3 5 5 3 5 1 4 5 1 0 5 5 4 1 5 4 0 4 4 5 3 4 3 4 0 4 1 3 1 4 4 1 1
 3 4 1 4 1 0 3 5 1 5 4 2 3 5 1 0 1 0 5 3 5 5 5 1 4 1 1 1 0 1 1 0 2 5 5 0 1
 4 4 5 4 1 4 4 5 5 0 3 1 1 5 0 0 5 0 0 5 1 5 5 1 3 1 5 0 0 4 4 4 5 0 4 1 0
 0 1 4 1 1 1 5 2 1 1 5 5 4 4 4 0 5 1 4 3 2 5 5 4 4 3 5 0 1 5 3 5 3 1 0 5 1
 3 5 2 1 1 4 3 0 5 5 2 0 0 1 0 1 4 1 3]
Classes after: [0 1 2 3 4 5]
Y_pred: [5 4 5 0 0 5 1 3 3 5 1 1 1 1 0 1 4 3 5 5 5 1 4 3 1 4 5 1 4 1 3 4 0 0 0 5 5
 4 3 1 2 0 1 3 4 4 3 3 3 5 3 5 5 4 1 5 0 1 1 0 5 3 0 5 1 1 1 1]
y_val [5 4 5 1 0 5 1 3 3 5 1 2 1 1 0 1 1 3 5 5 5 1 4 4 1 4 5 1 4 1 0 4 0 0 0 5 5
 4 1 1 2 0 1 3 4 4 1 1 3 5 3 5 5 4 1 5 0 1 1 0 5 3 0 5 1 1 1 1]
y_pred before save =  
 [[4.89336671e-05 5.59508123e-07 3.35017063e-07 1.33323520e-05
  3.58677653e-06 9.99933243e-01]
 [4.59373041e-05 4.98067720e-07 7.97402572e-06 1.22166864e-04
  9.99591529e-01 2.31834420e

1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 946ms/step

2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 895ms/step
Classes before: [4 1 5 0 3 5 0 5 1 4 0 5 0 4 0 0 5 3 4 5 5 5 4 3 1 2 4 4 0 1 5 5 1 0 4 5 3
 3 1 4 4 3 5 5 3 5 1 4 5 1 0 5 5 4 1 5 4 0 4 4 5 3 4 3 4 0 4 1 3 1 4 4 1 1
 3 4 1 4 1 0 3 5 1 5 4 2 3 5 1 0 1 0 5 3 5 5 5 1 4 1 1 1 0 1 1 0 2 5 5 0 1
 4 4 5 4 1 4 4 5 5 0 3 1 1 5 0 0 5 0 0 5 1 5 5 1 3 1 5 0 0 4 4 4 5 0 4 1 0
 0 1 4 1 1 1 5 2 1 1 5 5 4 4 4 0 5 1 4 3 2 5 5 4 4 3 5 0 1 5 3 5 3 1 0 5 1
 3 5 2 1 1 4 3 0 5 5 2 0 0 1 0 1 4 1 3]
Classes after: [0 1 2 3 4 5]
Y_pred: [5 4 5 1 0 5 1 3 3 5 4 2 1 1 0 1 4 3 5 5 5 1 4 4 1 4 5 1 4 1 3 4 0 0 0 5 5
 4 4 1 2 0 1 3 4 4 1 1 3 5 3 5 5 4 1 5 0 1 1 0 5 3 0 5 1 1 1 1]
y_val [5 4 5 1 0 5 1 3 3 5 1 2 1 1 0 1 1 3 5 5 5 1 4 4 1 4 5 1 4 1 0 4 0 0 0 5 5
 4 1 1 2 0 1 3 4 4 1 1 3 5 3 5 5 4 1 5 0 1 1 0 5 3 0 5 1 1 1 1]
y_pred before save =  
 [[1.72553671e-04 1.86161424e-06 6.27118823e-07 1.36762060e-06
  9.20310413e-06 9.99814451e-01]
 [2.46872169e-05 3.20739559e-07 1.97003078e-06 1.52609544e-04
  9.99655128e-01 1.65324673e

2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step
Classes before: [4 1 5 0 3 5 0 5 1 4 0 5 0 4 0 0 5 3 4 5 5 5 4 3 1 2 4 4 0 1 5 5 1 0 4 5 3
 3 1 4 4 3 5 5 3 5 1 4 5 1 0 5 5 4 1 5 4 0 4 4 5 3 4 3 4 0 4 1 3 1 4 4 1 1
 3 4 1 4 1 0 3 5 1 5 4 2 3 5 1 0 1 0 5 3 5 5 5 1 4 1 1 1 0 1 1 0 2 5 5 0 1
 4 4 5 4 1 4 4 5 5 0 3 1 1 5 0 0 5 0 0 5 1 5 5 1 3 1 5 0 0 4 4 4 5 0 4 1 0
 0 1 4 1 1 1 5 2 1 1 5 5 4 4 4 0 5 1 4 3 2 5 5 4 4 3 5 0 1 5 3 5 3 1 0 5 1
 3 5 2 1 1 4 3 0 5 5 2 0 0 1 0 1 4 1 3]
Classes after: [0 1 2 3 4 5]
Y_pred: [5 4 5 1 0 5 1 3 3 5 1 2 1 1 0 1 4 3 5 5 5 0 4 4 1 4 5 1 4 1 3 4 0 0 0 5 5
 4 1 1 2 0 1 3 4 4 1 1 3 5 3 5 5 4 1 5 0 1 1 0 5 3 0 5 1 1 1 1]
y_val [5 4 5 1 0 5 1 3 3 5 1 2 1 1 0 1 1 3 5 5 5 1 4 4 1 4 5 1 4 1 0 4 0 0 0 5 5
 4 1 1 2 0 1 3 4 4 1 1 3 5 3 5 5 4 1 5 0 1 1 0 5 3 0 5 1 1 1 1]
y_pred before save =  
 [[2.80196837e-04 3.16659243e-05 1.98931207e-06 1.26773848e-05
  5.08576204e-06 9.99668360e-01]
 [4.28755338e-05 9.72529051e-07 1.08248660e-05 2.02602023e-04
  9.99509692e-01 2.33001454e-04

2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 878ms/step
Classes before: [4 1 5 0 3 5 0 5 1 4 0 5 0 4 0 0 5 3 4 5 5 5 4 3 1 2 4 4 0 1 5 5 1 0 4 5 3
 3 1 4 4 3 5 5 3 5 1 4 5 1 0 5 5 4 1 5 4 0 4 4 5 3 4 3 4 0 4 1 3 1 4 4 1 1
 3 4 1 4 1 0 3 5 1 5 4 2 3 5 1 0 1 0 5 3 5 5 5 1 4 1 1 1 0 1 1 0 2 5 5 0 1
 4 4 5 4 1 4 4 5 5 0 3 1 1 5 0 0 5 0 0 5 1 5 5 1 3 1 5 0 0 4 4 4 5 0 4 1 0
 0 1 4 1 1 1 5 2 1 1 5 5 4 4 4 0 5 1 4 3 2 5 5 4 4 3 5 0 1 5 3 5 3 1 0 5 1
 3 5 2 1 1 4 3 0 5 5 2 0 0 1 0 1 4 1 3]
Classes after: [0 1 2 3 4 5]
Y_pred: [5 4 5 1 0 5 1 3 3 5 1 2 1 1 0 1 4 3 5 5 5 1 4 4 1 4 5 1 4 1 3 4 0 0 0 5 5
 4 4 1 2 0 1 3 4 4 1 1 3 5 3 5 5 4 1 5 0 1 1 0 5 3 0 5 1 1 1 1]
y_val [5 4 5 1 0 5 1 3 3 5 1 2 1 1 0 1 1 3 5 5 5 1 4 4 1 4 5 1 4 1 0 4 0 0 0 5 5
 4 1 1 2 0 1 3 4 4 1 1 3 5 3 5 5 4 1 5 0 1 1 0 5 3 0 5 1 1 1 1]
y_pred before save =  
 [[9.11044572e-06 5.89438223e-08 1.13110566e-07 3.96589058e-07
  6.60323280e-08 9.99990225e-01]
 [1.72349610e-05 1.38598182e-07 1.96224082e-05 5.38779350e-05
  9.99792278e-01 1.16970958e

2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 908ms/step
Classes before: [4 1 5 0 3 5 0 5 1 4 0 5 0 4 0 0 5 3 4 5 5 5 4 3 1 2 4 4 0 1 5 5 1 0 4 5 3
 3 1 4 4 3 5 5 3 5 1 4 5 1 0 5 5 4 1 5 4 0 4 4 5 3 4 3 4 0 4 1 3 1 4 4 1 1
 3 4 1 4 1 0 3 5 1 5 4 2 3 5 1 0 1 0 5 3 5 5 5 1 4 1 1 1 0 1 1 0 2 5 5 0 1
 4 4 5 4 1 4 4 5 5 0 3 1 1 5 0 0 5 0 0 5 1 5 5 1 3 1 5 0 0 4 4 4 5 0 4 1 0
 0 1 4 1 1 1 5 2 1 1 5 5 4 4 4 0 5 1 4 3 2 5 5 4 4 3 5 0 1 5 3 5 3 1 0 5 1
 3 5 2 1 1 4 3 0 5 5 2 0 0 1 0 1 4 1 3]
Classes after: [0 1 2 3 4 5]
Y_pred: [5 4 5 1 0 5 1 3 3 5 1 1 1 1 0 1 4 3 5 5 5 1 4 4 1 4 5 1 4 1 3 4 0 0 0 5 5
 4 1 1 2 0 1 3 4 4 1 1 3 5 3 5 5 4 1 5 0 1 1 0 5 3 0 5 1 1 1 1]
y_val [5 4 5 1 0 5 1 3 3 5 1 2 1 1 0 1 1 3 5 5 5 1 4 4 1 4 5 1 4 1 0 4 0 0 0 5 5
 4 1 1 2 0 1 3 4 4 1 1 3 5 3 5 5 4 1 5 0 1 1 0 5 3 0 5 1 1 1 1]
y_pred before save =  
 [[7.95762971e-05 5.03611300e-05 2.13929798e-06 6.71480439e-06
  5.02706314e-07 9.99860644e-01]
 [1.81767391e-05 1.09278437e-06 3.71465212e-05 1.97956790e-04
  9.99339521e-01 4.06071922e

In [ ]:
import os
import pandas as pd
import numpy as np

# Função auxiliar para criar o caminho
def construct_path(*args):
    return os.path.join(*args)

# Função auxiliar para processar métricas de um dataset
def process_dataset(clf_name, archive_name, dataset_name, root_dir, path):
    # Construção do caminho do arquivo
    output_dir = construct_path(root_dir, archive_name, dataset_name, 'df_metrics_val.csv')
    # Verifica se o caminho existe
    if not os.path.exists(output_dir):
        print(f"Caminho {output_dir} não encontrado. Pulando...")
        return None

    try:
        # Lê os dados
        df_metrics = pd.read_csv(output_dir)
    except Exception as e:
        print(f"Erro ao ler {output_dir}: {e}")
        return None

    print(f"Arquivo lido: {output_dir}")

    # Coleta as métricas
    return {
        'classifier_name': clf_name,
        'archive_name': archive_name,
        'dataset_name': dataset_name,
        'precision': df_metrics['precision'].mean(),
        'recall': df_metrics['recall'].mean(),
        'f1_score': df_metrics['f1_score'].mean(),
        'accuracy': df_metrics['accuracy'].mean(),
        'specificity': df_metrics['specificity'].mean(),
        'duration': df_metrics['duration'].mean(),
        'std_f1_score': np.std(df_metrics['f1_score']),
        'std_accuracy': np.std(df_metrics['accuracy']),
        'std_specificity': np.std(df_metrics['specificity']),
    }

# Função principal para gerar o CSV
def gen_results_csv(clf_name, path, root_dir):
    # Nomes das pastas e datasets
    ARCHIVE_NAMES = [
        'TSC', 'TSC_itr_1', 'TSC_itr_2', 'TSC_itr_3', 'TSC_itr_4',
        'TSC_itr_5', 'TSC_itr_6', 'TSC_itr_7', 'TSC_itr_8', 'TSC_itr_9'
    ]
    # UNIVARIATE_DATASET_NAMES = ['AllCandidasBloadOversample']

    results = []

    # Iteração sobre os arquivos e datasets
    for archive_index, archive_name in enumerate(ARCHIVE_NAMES):
        for dataset_index, dataset_name in enumerate(UNIVARIATE_DATASET_NAMES):
            result = process_dataset(clf_name, archive_name, dataset_name, root_dir, path)
            if result:
                result.update({
                    'ord1': dataset_index,
                    'ord2': archive_index,
                    'iteration': archive_index,
                })
                results.append(result)

    # Cria o DataFrame final
    res_df = pd.DataFrame(results)

    # Ordena o DataFrame
    res_df = res_df.sort_values(by=['ord2', 'ord1']).reset_index(drop=True)

    return res_df

# Configuração dos parâmetros
root_dir = '/content/drive/MyDrive/Colab Notebooks/Candidas/results/inception/'
clf_name = 'inception'
path = '/'

# Geração do CSV
res = gen_results_csv(clf_name, path, root_dir)
print(res)

# Salvando o resultado em um arquivo CSV
output_file = construct_path(root_dir, f"{clf_name}.csv")
res.to_csv(output_file, index=False)
print(f"Arquivo salvo em: {output_file}")


Arquivo lido: /content/drive/MyDrive/Colab Notebooks/Candidas/results/inception/TSC/AllCandidasBload/df_metrics_val.csv
Arquivo lido: /content/drive/MyDrive/Colab Notebooks/Candidas/results/inception/TSC_itr_1/AllCandidasBload/df_metrics_val.csv
Arquivo lido: /content/drive/MyDrive/Colab Notebooks/Candidas/results/inception/TSC_itr_2/AllCandidasBload/df_metrics_val.csv
Arquivo lido: /content/drive/MyDrive/Colab Notebooks/Candidas/results/inception/TSC_itr_3/AllCandidasBload/df_metrics_val.csv
Arquivo lido: /content/drive/MyDrive/Colab Notebooks/Candidas/results/inception/TSC_itr_4/AllCandidasBload/df_metrics_val.csv
Arquivo lido: /content/drive/MyDrive/Colab Notebooks/Candidas/results/inception/TSC_itr_5/AllCandidasBload/df_metrics_val.csv
Arquivo lido: /content/drive/MyDrive/Colab Notebooks/Candidas/results/inception/TSC_itr_6/AllCandidasBload/df_metrics_val.csv
Arquivo lido: /content/drive/MyDrive/Colab Notebooks/Candidas/results/inception/TSC_itr_7/AllCandidasBload/df_metrics_val.cs

In [ ]:
import os
import pandas as pd
import numpy as np

# Função auxiliar para criar o caminho
def construct_path(*args):
    return os.path.join(*args)

# Função auxiliar para processar métricas de um dataset
def process_dataset(clf_name, archive_name, dataset_name, root_dir):
    # Construção do caminho do arquivo
    output_dir = construct_path(root_dir, archive_name, dataset_name, 'all_class_metrics.csv')

    # Verifica se o caminho existe
    if not os.path.exists(output_dir):
        print(f"Caminho {output_dir} não encontrado. Pulando...")
        return None

    try:
        # Lê os dados
        df_metrics = pd.read_csv(output_dir)
    except Exception as e:
        print(f"Erro ao ler {output_dir}: {e}")
        return None

    print(f"Arquivo lido: {output_dir}")

    # Calcula a média das métricas por classe entre os folds
    grouped_metrics = df_metrics.groupby('class').mean(numeric_only=True).reset_index()

    # Adiciona informações adicionais (se necessário)
    grouped_metrics['archive_name'] = archive_name
    grouped_metrics['dataset_name'] = dataset_name
    grouped_metrics['classifier_name'] = clf_name

    return grouped_metrics

# Função principal para gerar o CSV consolidado
def gen_results_csv(clf_name, root_dir):
    # Nomes das pastas e datasets
    ARCHIVE_NAMES = [
        'TSC', 'TSC_itr_1', 'TSC_itr_2', 'TSC_itr_3', 'TSC_itr_4',
        'TSC_itr_5', 'TSC_itr_6', 'TSC_itr_7', 'TSC_itr_8', 'TSC_itr_9'
    ]
    # DATASET_NAME = ['AllCandidasBloadOversample']

    results = []

    # Iteração sobre os arquivos e datasets
    for archive_name in ARCHIVE_NAMES:
        for dataset_name in UNIVARIATE_DATASET_NAMES:
            result = process_dataset(clf_name, archive_name, dataset_name, root_dir)
            if result is not None:
                results.append(result)

    # Concatena todos os resultados em um único DataFrame
    if results:
        final_df = pd.concat(results, ignore_index=True)
    else:
        final_df = pd.DataFrame()  # Retorna DataFrame vazio caso não haja dados

    return final_df

# Configuração dos parâmetros
root_dir = '/content/drive/MyDrive/Colab Notebooks/Candidas/results/inception/'
clf_name = 'inception'

# Geração do CSV consolidado
res = gen_results_csv(clf_name, root_dir)
if not res.empty:
    print(res)

    # Salvando o resultado em um arquivo CSV
    output_file = construct_path(root_dir, f"{clf_name}_aggregated_metrics.csv")
    res.to_csv(output_file, index=False)
    print(f"Arquivo salvo em: {output_file}")
else:
    print("Nenhum dado encontrado para processar.")


Arquivo lido: /content/drive/MyDrive/Colab Notebooks/Candidas/results/inception/TSC/AllCandidasBload/all_class_metrics.csv
Arquivo lido: /content/drive/MyDrive/Colab Notebooks/Candidas/results/inception/TSC_itr_1/AllCandidasBload/all_class_metrics.csv
Arquivo lido: /content/drive/MyDrive/Colab Notebooks/Candidas/results/inception/TSC_itr_2/AllCandidasBload/all_class_metrics.csv
Arquivo lido: /content/drive/MyDrive/Colab Notebooks/Candidas/results/inception/TSC_itr_3/AllCandidasBload/all_class_metrics.csv
Arquivo lido: /content/drive/MyDrive/Colab Notebooks/Candidas/results/inception/TSC_itr_4/AllCandidasBload/all_class_metrics.csv
Arquivo lido: /content/drive/MyDrive/Colab Notebooks/Candidas/results/inception/TSC_itr_5/AllCandidasBload/all_class_metrics.csv
Arquivo lido: /content/drive/MyDrive/Colab Notebooks/Candidas/results/inception/TSC_itr_6/AllCandidasBload/all_class_metrics.csv
Arquivo lido: /content/drive/MyDrive/Colab Notebooks/Candidas/results/inception/TSC_itr_7/AllCandidasBl

In [ ]:
import os
import pandas as pd

def construct_path(*args):
    """
    Constrói um caminho usando `os.path.join`.
    """
    return os.path.join(*args)


def process_dataset(clf_name, archive_name, dataset_name, root_dir):
    """
    Processa métricas de um dataset específico, coletando as métricas por classe.

    Args:
        clf_name (str): Nome do classificador.
        archive_name (str): Nome do arquivo de métricas.
        dataset_name (str): Nome do dataset.
        root_dir (str): Diretório raiz dos resultados.

    Returns:
        pd.DataFrame | None: DataFrame com métricas por classe ou None se o arquivo não for encontrado.
    """
    output_dir = construct_path(root_dir, archive_name, dataset_name, 'all_class_metrics.csv')

    if not os.path.exists(output_dir):
        print(f"[AVISO] Caminho {output_dir} não encontrado. Pulando...")
        return None

    try:
        df_metrics = pd.read_csv(output_dir)
        print(f"[INFO] Arquivo lido com sucesso: {output_dir}")
    except Exception as e:
        print(f"[ERRO] Falha ao ler {output_dir}: {e}")
        return None

    # Retorna os dados, mantendo as informações de classe
    df_metrics['archive_name'] = archive_name
    df_metrics['dataset_name'] = dataset_name
    df_metrics['classifier_name'] = clf_name

    return df_metrics


def gen_results_csv(clf_name, root_dir, archive_names=None, dataset_names=None):
    """
    Consolida métricas de diferentes datasets, calcula médias por classe e gera um DataFrame final.

    Args:
        clf_name (str): Nome do classificador.
        root_dir (str): Diretório raiz dos resultados.
        archive_names (list | None): Lista de arquivos a serem processados.
        dataset_names (list | None): Lista de datasets a serem processados.

    Returns:
        pd.DataFrame: DataFrame consolidado com métricas médias por classe.
    """
    if archive_names is None:
        archive_names = [
            'TSC', 'TSC_itr_1', 'TSC_itr_2', 'TSC_itr_3',
            'TSC_itr_4', 'TSC_itr_5', 'TSC_itr_6', 'TSC_itr_7',
            'TSC_itr_8', 'TSC_itr_9'
        ]

    # if dataset_names is None:
    #     dataset_names = ['AllCandidasBloadOversample']
    # if UNIVARIATE_DATASET_NAMES is None:
    #      UNIVARIATE_DATASET_NAMES = ['AllCandidasBload']
    all_metrics = []
    for archive_name in archive_names:
        for dataset_name in UNIVARIATE_DATASET_NAMES:
            result = process_dataset(clf_name, archive_name, dataset_name, root_dir)
            if result is not None:
                all_metrics.append(result)

    # Concatena todos os dados em um único DataFrame
    if all_metrics:
        all_data = pd.concat(all_metrics, ignore_index=True)

        # Calcula a média das métricas por classe
        metrics_mean = (
            all_data
            .groupby('class')
            .mean(numeric_only=True)
            .reset_index()
        )
    else:
        print("[AVISO] Nenhum dado encontrado para processar.")
        metrics_mean = pd.DataFrame()

    return metrics_mean


if __name__ == "__main__":
    # Configurações
    ROOT_DIR = '/content/drive/MyDrive/Colab Notebooks/Candidas/results/inception/'
    CLF_NAME = 'inception'

    # Geração do CSV consolidado
    metrics_by_class = gen_results_csv(CLF_NAME, ROOT_DIR)
    if not metrics_by_class.empty:
        # Salvando o resultado em um arquivo CSV
        output_file = construct_path(ROOT_DIR, f"{CLF_NAME}_class_average_metrics.csv")
        metrics_by_class.to_csv(output_file, index=False)
        print(f"[SUCESSO] Arquivo salvo em: {output_file}")
    else:
        print("[ERRO] Nenhum resultado gerado.")


[INFO] Arquivo lido com sucesso: /content/drive/MyDrive/Colab Notebooks/Candidas/results/inception/TSC/AllCandidasBload/all_class_metrics.csv
[INFO] Arquivo lido com sucesso: /content/drive/MyDrive/Colab Notebooks/Candidas/results/inception/TSC_itr_1/AllCandidasBload/all_class_metrics.csv
[INFO] Arquivo lido com sucesso: /content/drive/MyDrive/Colab Notebooks/Candidas/results/inception/TSC_itr_2/AllCandidasBload/all_class_metrics.csv
[INFO] Arquivo lido com sucesso: /content/drive/MyDrive/Colab Notebooks/Candidas/results/inception/TSC_itr_3/AllCandidasBload/all_class_metrics.csv
[INFO] Arquivo lido com sucesso: /content/drive/MyDrive/Colab Notebooks/Candidas/results/inception/TSC_itr_4/AllCandidasBload/all_class_metrics.csv
[INFO] Arquivo lido com sucesso: /content/drive/MyDrive/Colab Notebooks/Candidas/results/inception/TSC_itr_5/AllCandidasBload/all_class_metrics.csv
[INFO] Arquivo lido com sucesso: /content/drive/MyDrive/Colab Notebooks/Candidas/results/inception/TSC_itr_6/AllCandid

In [ ]:
from pickle import dump
def save_results(df_to_save, name, train_val_test):
    print("Saving metrics...")
    print(f"Df accuracy: {df_to_save}")
    df_to_save.to_csv(root_dir+ '/df_'+name+'_'+train_val_test+'2.csv', index=False)
    print("Accuracy of models:")
    print(df_to_save)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
import time
def test_final_model(x_val, y_val, model):
    scoring = ['accuracy','precision_macro', 'recall_macro','f1_macro']
#     for name, model in models:
        # df_test = cross_validate(model, X_test, y_test, cv=kf, scoring=scoring
        #                          , return_train_score=True)
#         print(f'Model in test: {name}')

    df_test = pd.DataFrame([], columns=['accuracy','recall','precision','f1_score','specificity','testTime'])

    start_time = time.time()
    y_pred = model.predict(x_val, batch_size=1500)
    time_prediction = time.time()-start_time

    y_pred = np.argmax(y_pred, axis=1)
    onehot_encoder = OneHotEncoder()
    y_pred = y_pred.reshape(len(y_pred), 1)
    y_pred = onehot_encoder.fit_transform(y_pred)
    print("Y pred ---- ")
    print(y_pred)
    print("Y val")
    print(y_val)
    accuracy  =  accuracy_score(y_val, y_pred)
    recall    =  recall_score(y_val, y_pred, average='macro')
    precision =  precision_score(y_val, y_pred, average='macro')
    f1  =  f1_score(y_val, y_pred, average='macro')
    specificity = specificity_score(y_val, y_pred)
    print(f"accuracy: {accuracy}, recall: {recall}, precision: {precision},f1: {f1} ")
    df_test.at[0, 'accuracy'] = accuracy
    df_test.at[0, 'recall'] = recall
    df_test.at[0,'precision'] = precision
    df_test.at[0,'f1_score'] = f1
    df_test.at[0,'specificity'] = specificity
    df_test.at[0,'testTime'] =  time_prediction
    print(df_test.keys())
    save_results(df_test,"INCEPTION", 'test')
    print("Finished test!")

In [ ]:
archive_name = 'TSC_itr_3'
tmp_output_directory = root_dir + '/'+ archive_name + '/'
output_directory = tmp_output_directory + dataset_name + '/'
print(tmp_output_directory)
# dataset = np.load(output_directory+'/y_pred'+'.npy', encoding='bytes')
# dataset

/content/drive/MyDrive/Colab Notebooks/Candidas/results/inception//TSC_itr_3/


In [ ]:
print(x_val.shape)
print(y_val.shape)

(68, 512, 1)
(68, 6)


In [ ]:
model_path = output_directory + 'best_model.keras'
model = keras.models.load_model(model_path)
test_final_model(x_val, y_val, model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Y pred ---- 
  (0, 5)	1.0
  (1, 0)	1.0
  (2, 4)	1.0
  (3, 3)	1.0
  (4, 3)	1.0
  (5, 1)	1.0
  (6, 1)	1.0
  (7, 5)	1.0
  (8, 4)	1.0
  (9, 1)	1.0
  (10, 0)	1.0
  (11, 4)	1.0
  (12, 0)	1.0
  (13, 3)	1.0
  (14, 4)	1.0
  (15, 1)	1.0
  (16, 5)	1.0
  (17, 3)	1.0
  (18, 1)	1.0
  (19, 3)	1.0
  (20, 5)	1.0
  (21, 1)	1.0
  (22, 1)	1.0
  (23, 4)	1.0
  (24, 3)	1.0
  :	:
  (43, 3)	1.0
  (44, 5)	1.0
  (45, 3)	1.0
  (46, 4)	1.0
  (47, 3)	1.0
  (48, 3)	1.0
  (49, 1)	1.0
  (50, 1)	1.0
  (51, 1)	1.0
  (52, 3)	1.0
  (53, 5)	1.0
  (54, 5)	1.0
  (55, 1)	1.0
  (56, 1)	1.0
  (57, 5)	1.0
  (58, 0)	1.0
  (59, 0)	1.0
  (60, 1)	1.0
  (61, 1)	1.0
  (62, 4)	1.0
  (63, 5)	1.0
  (64, 2)	1.0
  (65, 3)	1.0
  (66, 5)	1.0
  (67, 3)	1.0
Y val
[[0. 0. 0. 0. 0. 1.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0